###    kw3Testing.ipynb
####       January 1 2023

####   Development and Testing of function to perform 
####     Kruskall-Wallace non-parametric anova ( roughly speaking, anova on ranked data)


#### Imports

In [1]:
import pandas as pd
import numpy as np

####  sample (columns) by features (rows) dataset
####  missing (NaN) values are allowed--will be ignored in calculations, including group sizes

In [2]:
dataf = pd.read_table('kw3TestingData.txt', na_values=' ').set_index('UID')
print(dataf)

        S1  S2  S3  S4    S5    S6    S7  S8   S9  S10  ...   S19   S20  S21  \
UID                                                     ...                    
ev01   0.0   1   2   3   4.0   5.0   6.0   7  8.0  9.0  ...   5.5   6.0    7   
ev02   0.0   0   2   2   4.0   4.0   6.0   7  8.0  9.0  ...   4.5   6.0    7   
ev03   0.0   1   2   3   4.0   5.0   6.0   7  NaN  NaN  ...   5.5   6.0    7   
ev04  17.0  16  15  14  13.0  12.0  11.0  10  9.0  8.0  ...  12.5  11.0   10   
ev05   NaN  16  15  14  13.0  12.0  11.0  10  9.0  8.0  ...  12.5  11.0   10   
ev06   1.0   3   2   4   5.0   6.0   3.0   1  4.0  2.0  ...   6.5   3.0    1   
ev07  17.0   0   1   2   NaN   NaN   NaN   6  7.0  8.0  ...   NaN   NaN    6   
ev08   0.0   1   3   2   4.0   5.0   7.0   6  8.0  9.0  ...   5.5   7.0    6   

      S22  S23   S24  S25  S26  S27   S28  
UID                                        
ev01  8.0  9.0  10.0   11   12   13  14.5  
ev02  8.0  9.0  10.0   11   12   13  14.5  
ev03  NaN  NaN   NaN   

####  set up groups (of samples) information in DataFrame
####   the number of groups should be >= 3 (if 2 groups should use mwu)
####   for time being, assume group designation is provided for each sample in data matrix

In [3]:
#  Hardcode some sample grouping info for testing

groups = [1,2,3,4, 1,2,3,4, 1,2,3,4, 1,1,2,2, 3,4,4,4, 1,2,3,4, 1,2,3,4]
samps = list(dataf.columns)
#print(groups, samps)
grpdf = pd.DataFrame(zip(groups,samps),index=samps,columns=['grp','samp'])
display(grpdf)


,grp,samp
S1,1,S1
S2,2,S2
S3,3,S3
S4,4,S4
S5,1,S5
S6,2,S6
S7,3,S7
S8,4,S8
S9,1,S9
S10,2,S10


In [5]:
def kruskalWallisChecks( dataf, grpdf, *args, min_group_size=6 ): 
#
#   This function performs some checks before calling KruskalWallisCompute().    
#
#   dataf is a pandas data frame with samples as columns and events (features) as rows.  
#   Event names must be the row names (index), not in a column.
#   Missing values (np.nan='NaN') are allowed
#
#   grpsdf is a pandas data frame with a group membership column ('grp' for now ).
#   Sample names must be the row names (index), not in a column.
#   Aside from ordering, must be 1-1 match between dataf.columns and grpdf.index
#
#   min_group_size is minimum group size for kw test to be computed.
#   min_group_size = 6 is based on statistical rule-of-thumb for Mann-Whitney 2-group test.
#
#  Returns integer to indicate course of action:
#     0 = Problem, do not run KWTest
#     1 = Run KW test
#     2 = Two-sample situation, run MWU test instead

    import pandas as pd
    import numpy as np
    
    # --- Check for 1-1 match between sample names provided in grpdf and the data matrix

    if set(dataf.columns)!= set(grpdf.index) :
        print('Group information is inconsistent with names in data matrix')
        return( 0 )
  
        
    grpCounts = grpdf['grp'].value_counts().to_frame()  # returns a pandas df of counts, indexed by grp, decreasing by count
    grpCounts.columns=['grpRawN']
    nGrps = grpCounts.shape[0]
    minGrpN = min( grpCounts['grpRawN'] )
    # print('nGrps, minGrpN',nGrps,minGrpN)
    
    
    # -- Handle 2 or fewer groups --

    if nGrps < 2 :
        print('Number of sample groups is < 2; Kruskal-Wallis test not conducted')
        return( 0 )
        
    if nGrps == 2 :
        print('Only 2 sample groups found, switching to Mann-Whitney-U test')
        return( 2 )
    
    # -- Don't proceed if already know a group size is < minimum --   

    if minGrpN < min_group_size:
        print('Kruskal-Wallis test not conducted: 1 or more groups has fewer samples than minimum group size: ',minGrpN,' < ',min_group_size)
        return( 0 )
    
    return( 1 )
    

In [7]:
from kw3 import kruskalWallisChecks

###  Run the Checks function to verify desired behavior

In [23]:
# All criteria met
runKW = kruskalWallisChecks( dataf, grpdf, min_group_size=6 )
print(runKW) 
# Should run w/ runKW == 1  

# Test 1 group situation
grpdf_1 = grpdf.copy()
grpdf_1['grp'] =1
display(grpdf)
runKW = kruskalWallisChecks( dataf, grpdf_1, min_group_size=6 )
print(runKW)
# Should return runKW = 0 & groups < 2 message

# Test 2 group situation
grpdf_2 = grpdf.copy()
grpdf_2['grp'] = [1,2,2,2, 1,2,2,2, 1,2,2,2, 1,1,2,2, 1,1,2,2, 1,2,2,2, 1,2,2,2]
display(grpdf_2)
runKW = kruskalWallisChecks( dataf, grpdf_2, min_group_size=6 )
print(runKW)
# Should return runKW = 0 & groups < 2 message


# Test behavior if a group size is < minimum 
runKW = kruskalWallisChecks( dataf, grpdf, min_group_size=10 )
print(runKW) 
# Should return runKW == 0  & fewer samples than min message


# Test sample name mismatch
grpdf_sn = grpdf.copy()
grpdf_sn.at['S1','samp'] = 'S100'
grpdf_sn.rename(index={'S1':'S100'},inplace=True)
display(grpdf_sn)
runKW = kruskalWallisChecks( dataf, grpdf_sn, min_group_size=6 )
print(runKW)
# Should return runKW = 0 & groups < 2 message

1


,grp,samp
S1,1,S1
S2,2,S2
S3,3,S3
S4,4,S4
S5,1,S5
S6,2,S6
S7,3,S7
S8,4,S8
S9,1,S9
S10,2,S10


Number of sample groups is < 2; Kruskal-Wallis test not conducted
0


,grp,samp
S1,1,S1
S2,2,S2
S3,2,S3
S4,2,S4
S5,1,S5
S6,2,S6
S7,2,S7
S8,2,S8
S9,1,S9
S10,2,S10


Only 2 sample groups found, switching to Mann-Whitney-U test
2
Kruskal-Wallis test not conducted: 1 or more groups has fewer samples than minimum group size:  6  <  10
0


,grp,samp
S100,1,S100
S2,2,S2
S3,3,S3
S4,4,S4
S5,1,S5
S6,2,S6
S7,3,S7
S8,4,S8
S9,1,S9
S10,2,S10


Group information is inconsistent with names in data matrix
0


In [ ]:
# Could explore TRY structure  EXCEPT exception (possibly use pass here) 

#if runKW == 0:
    # stop
#    pass
#elif runKW == 2:
    # call MWU
#    pass
#else:
    #resKW = KruskalWallisCompute(dataf, grpdf, min_group_size=6 )
#    pass

In [24]:
def KruskalWallisCompute(dataf, grpdf, *args, min_group_size=6 ): 
    #
    #   This function performs the Kruskal-Wallis NP ANOVA test.    
    #
    #   dataf is a pandas data frame with samples as columns and events (features) as rows.  
    #   Event names must be the row names (index), not in a column.
    #   Missing values (np.nan='NaN') are allowed
    #
    #   grpsdf is a pandas data frame with a group membership column.
    #   Sample names must be the row names (index), not in a column.
    #   Aside from ordering, must be 1-1 match between dataf.columns and grpdf.index
    #
    #   min_group_size is minimum group size for kw test to be computed.
    #   6 is based on statistical rule-of-thumb for Mann-Whitney 2-group test.
    #
    #   Returns pandas dataframe with columns containing group Ns (xNaN) & medians, value of KW test statistic, and p-value.
    #   Returned df has row for each event, even if test was not computed.
    
    import pandas as pd
    import numpy as np

    import scipy.stats
    

    # Set up little df with grp info, for group Ns and medians work
    
    grpCounts = grpdf['grp'].value_counts().to_frame()  # returns a pandas df of counts, indexed by grp, decreasing by count
    grpCounts.columns=['grpRawN']
    grpCounts['grpID']=grpCounts.index
    grpCounts = grpCounts.sort_index()
    # print(grpCounts) 
    nGrps = grpCounts.shape[0]

    # Compute group N and median for each event, in blocks by group, right-side joining as go. 
    #. Accumulate in resdf
    gindex = 0
    for gID in grpCounts.index:
        gindex = gindex + 1
        gSamps = grpdf.loc[grpdf['grp'] == gID, 'samp'].tolist()
    #    print(gSamps)
        gdf=dataf[gSamps]
    #    display(gdf)
    #
        meds = np.nanmedian(gdf,axis=1)
    #    print(meds)

        okVals = np.sum(~np.isnan(gdf), axis=1)
    #    print(okVals)

        tempdf=pd.DataFrame(zip(okVals,meds),index=okVals.index)
        tempdf.columns=[('N_' + str(gID)),('Median_' + str(gID))]
        if gindex == 1:
            resdf = tempdf.copy()
        else:
            resdf = resdf.set_index(resdf.index).join(tempdf)
 #       display(resdf)
# display(resdf) 
#    return( resdf )

#    Add columns for the kw statistic & p-value 
    
    resdf['kwStat'], resdf['kwPval'] = [ np.nan, np.nan ]
    

#   ID and extract data for events that meet the min_group_size criterion

    colNums = list(range(0,2*nGrps,2))
    Nsdf = resdf.take(colNums, axis=1)
    # display(Nsdf)
    evMins = Nsdf.min(axis=1)
    evKeeps = evMins.index[evMins >= min_group_size]  
    evKeeps = evKeeps.to_list()
#    print(evKeeps)
    
#   Handle situation of no events meeting min_group_size criterion
    if len(evKeeps) == 0:
        print('Kruskal-Wallis test not conducted: No events meet minimum group size criterion.')
        return( resdf)

#   Subset data to events for which kw test will be conducted.   
    kwdata = dataf.loc[evKeeps,:]
#    display(kwdata)

#   Set up objects for the call to scipy.stats.kruskal()   
    y = np.array(kwdata)
    label, idx = np.unique(grpdf['grp'], return_inverse=True)
    groups = [y[0,idx == i] for i, l in enumerate(label)]

    
#  Perform KW test for each event in kwdata
    for indx in range(0, len(kwdata) ):
        ev = kwdata.index[indx]
        groups = [y[indx,idx == i] for i, l in enumerate(label)] 
        resdf.loc[ev,'kwStat'], resdf.loc[ev,'kwPval'] = scipy.stats.kruskal(*groups)

    return( resdf )
    

In [8]:
from kw3 import KruskalWallisCompute

In [9]:
# Run as intended

resdf = KruskalWallisCompute(dataf, grpdf, min_group_size=6 )
display(resdf)

,N_1,Median_1,N_2,Median_2,N_3,Median_3,N_4,Median_4,kwStat,kwPval
UID,,,,,,,,,,
ev01,7,8.0,7,9.0,6,9.5,8,8.5,0.358984,0.948579
ev02,7,8.0,7,9.0,6,9.5,8,8.5,0.309716,0.958191
ev03,6,9.0,5,12.0,4,9.5,7,7.0,NaN,NaN
ev04,7,9.0,7,8.0,6,7.5,8,8.5,0.393895,0.941501
ev05,6,7.5,7,8.0,6,7.5,7,10.0,NaN,NaN
ev06,7,5.0,7,6.0,6,4.5,8,5.5,0.858058,0.835536
ev07,6,10.5,5,8.0,4,8.5,6,9.5,NaN,NaN
ev08,7,6.0,7,8.0,6,8.0,8,6.0,1.005872,0.799831


In [28]:
#	   N_1	Median_1	N_2	Median_2	N_3	Median_3	N_4	Median_4	kwStat	kwPval
#UID										
#ev01	7	8.0	         7	9.0	         6	9.5	           8	8.5	  0.358984	0.948579
#ev02	7	8.0	         7	9.0	         6	9.5	           8	8.5	  0.309716	0.958191
#ev03	6	9.0	         5	12.0	     4	9.5	           7	7.0	     NaN	NaN
#ev04	7	9.0	         7	8.0	         6	7.5	           8	8.5	  0.393895	0.941501
#ev05	6	7.5	         7	8.0	         6	7.5	           7	10.0	 NaN	NaN
#ev06	7	5.0	         7	6.0	         6	4.5	           8	5.5	  0.858058	0.835536
#ev07	6	10.5	     5	8.0	         4	8.5	           6	9.5	     NaN	NaN
#ev08	7	6.0	         7	8.0	         6	8.0	           8	6.0	  1.005872	0.799831

###  Check behavior if min_group_size=10 --- should run and all kwStat & kwPval values should be NaN

In [27]:
resdf = KruskalWallisCompute(dataf, grpdf, min_group_size=10 )
display(resdf)

Kruskal-Wallis test not conducted: No events meet minimum group size criterion.


,N_1,Median_1,N_2,Median_2,N_3,Median_3,N_4,Median_4,kwStat,kwPval
UID,,,,,,,,,,
ev01,7,8.0,7,9.0,6,9.5,8,8.5,NaN,NaN
ev02,7,8.0,7,9.0,6,9.5,8,8.5,NaN,NaN
ev03,6,9.0,5,12.0,4,9.5,7,7.0,NaN,NaN
ev04,7,9.0,7,8.0,6,7.5,8,8.5,NaN,NaN
ev05,6,7.5,7,8.0,6,7.5,7,10.0,NaN,NaN
ev06,7,5.0,7,6.0,6,4.5,8,5.5,NaN,NaN
ev07,6,10.5,5,8.0,4,8.5,6,9.5,NaN,NaN
ev08,7,6.0,7,8.0,6,8.0,8,6.0,NaN,NaN
